In [77]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random
import operator
import math

In [78]:
from  collections import defaultdict
genres = defaultdict(int)

In [79]:
directory = "C:/Users/hp/Documents/genres/genres"
nfile = open("dat",'wb')
i=0;

for gen in os.listdir(directory):
    path = directory + "/" + gen
    if os.path.isdir(path):
        print(path)
        i+=1
        genres[i] = gen
        for aud in os.listdir(path):
            sr,data = wav.read(path+"/"+aud)
            mfcc_feat = mfcc(data,sr,winlen = 0.020,appendEnergy = False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_mat = mfcc_feat.mean(0)
            feature = (mean_mat, covariance, i)
            pickle.dump(feature, nfile)
        
nfile.close()

C:/Users/hp/Documents/genres/genres/blues
C:/Users/hp/Documents/genres/genres/classical
C:/Users/hp/Documents/genres/genres/country
C:/Users/hp/Documents/genres/genres/disco
C:/Users/hp/Documents/genres/genres/hiphop
C:/Users/hp/Documents/genres/genres/jazz
C:/Users/hp/Documents/genres/genres/metal
C:/Users/hp/Documents/genres/genres/pop
C:/Users/hp/Documents/genres/genres/reggae
C:/Users/hp/Documents/genres/genres/rock


In [80]:
dataset = []
def loadDS(name, split, tr,te):
    with open("dat",'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for i in dataset:
        if random.random()<split:
            tr.append(i)
        else:
            te.append(i)
            
trainingSet = []
testSet = []
loadDS("dat",0.66,trainingSet, testSet)


In [16]:
print(trainingSet)

[(array([ 66.93164965,   0.71885345,  -3.26783036,   4.18120038,
        -8.05094092,   6.4482592 , -17.76851742,  14.09181008,
       -18.33253624,   3.68556046, -10.55074362,   2.88748511,
        -7.78808286]), array([[107.79187315, -31.41110033,  -9.22698714, -58.2604521 ,
        -26.47249497, -32.54536346, -49.98443347,  15.73243236,
        -17.98625091,  11.68895105,  25.22345732,   9.21246259,
          2.06370737],
       [-31.41110033,  67.24066775, -27.33646206,   2.99181433,
         -1.56782767,   5.71153147,  28.79682267, -15.11618225,
         13.93898012, -21.12518659,  -6.02999789,   0.8215573 ,
          0.35850908],
       [ -9.22698714, -27.33646206, 162.0214623 ,  12.28739935,
        -24.64214157,  44.25096999, -12.97002892,  14.05913759,
        -26.96955285,  15.68387893, -22.19458709,   8.72633776,
         15.43161094],
       [-58.2604521 ,   2.99181433,  12.28739935, 157.55731366,
         77.41096632,  40.12079458,  49.67785321, -28.11579524,
         30.4

In [37]:
print(trainingSet[0][2])

1


In [28]:
print(len(dataset))

1000


In [29]:
print(len(testSet))

350


In [81]:
def getNeighbours(tSet, ex, k):
    distances=[]
    for i in tSet:
        d = dist(i, ex, k) + dist(ex, i, k)
        distances.append((i[2],d))
    distances.sort(key=operator.itemgetter(1))
    neighbours =[]
    for i in range(k):
        neighbours.append(distances[i][0])
#     print(neighbours)
    return neighbours

In [82]:
def nearestgen(neighbours):
    genv = {}
    for i in neighbours:
#         temp = neighbours[i]
#         print(temp)
        if i in genv:
            genv[i]+=1;
        else:
            genv[i]=1;
    sorter = sorted(genv.items(),key = operator.itemgetter(1), reverse = True)
    return sorter[0][0]

In [83]:
def acc(testSet, pred):
    corr=0
    for i in range (len(testSet)):
        if testSet[i][-1]==pred[i]:
            corr+=1
    return 1.0*corr/len(testSet)

In [84]:
def dist(i1,i2,k):
    mm1=i1[0]
    cm1=i1[1]
    mm2=i2[0]
    cm2=i2[1]
    d = np.trace(np.dot(np.linalg.inv(cm2),cm1))
    d+= np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 ) 
    d+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    d-= k
    return d

In [85]:
prediction=[]
for i in testSet:
    prediction.append(nearestgen(getNeighbours(trainingSet, i, 7)))

print(acc(testSet,prediction))

0.6795252225519288


In [93]:
test_path = "C:/Users/hp/Documents/genres/test"
for i in os.listdir(test_path):
    sr,data = wav.read(test_path+"/"+i)
    mfcc_feat = mfcc(data,sr,winlen = 0.020,appendEnergy = False)
    covariance = np.cov(np.matrix.transpose(mfcc_feat))
    mean_mat = mfcc_feat.mean(0)
    feature = (mean_mat, covariance, 0)
    genre_id = nearestgen(getNeighbours(dataset, feature, 7))
    print(i," ",genres[genre_id])

1n.wav   reggae
2b.wav   jazz
3s.wav   country
